In [1]:
import pandas as pd
import matplotlib.pylab as plt

In [53]:
# 加载数据
file_path = '/Users/fanxinwei/Desktop/code/git_repo/dsmp-2024-group20/input_data/test_predictions_comparison(10).csv'
file_path_20 = '/Users/fanxinwei/Desktop/code/git_repo/dsmp-2024-group20/input_data/test_predictions_comparison(20).csv'
data = pd.read_csv(file_path)
data_20 = pd.read_csv(file_path_20)
# get Forecast of data_20 and insert it into data
data['Forecast_20'] = data_20['Forecast']
data.head(5)
# 0平稳1上升2下降

,time,Actual,Forecast,avg_price,Forecast_20
0,2025-05-27 16:28:10,0,0,134.750000,0
1,2025-05-27 16:28:11,0,0,133.714286,0
2,2025-05-27 16:28:12,0,0,132.250000,0
3,2025-05-27 16:28:13,0,0,130.650000,0
4,2025-05-27 16:28:14,0,0,132.961538,0


In [102]:
# get first 1000 rows and make it into 10 dataframes
data_list = []
periods = 10
for i in range(periods):
    data_list.append(data.iloc[i*100:(i+1)*100])

In [112]:
# 初始资金
initial_funds = 10000
# 资金
funds = initial_funds
# 股数
stocks = 0
# 买卖点
buy_sell_points = []

In [104]:
# 交易策略 
# 假设df是包含你数据的DataFrame
def trade_strategy(trade_data, funds, stocks, buy_sell_points):
    final_funds = 0
    profit = 0
    for index, row in trade_data.iterrows():
        forecast = row['Forecast']
        price = row['avg_price']
        
        if forecast == 1 and funds >= price:  # 有足够资金买入时
            # 买入一股股票
            stocks += 1
            funds -= price  # 更新剩余资金
            buy_sell_points.append((row['time'], 'buy'))
            
        elif forecast == 2 and stocks > 0:  # 有股票可卖出时
            # 卖出一股股票
            stocks -= 1
            funds += price  # 更新资金
            buy_sell_points.append((row['time'], 'sell'))
            
        # 计算最终资金，考虑剩余的股票价值
    final_funds = funds + stocks * trade_data.iloc[-1]['avg_price']
    profit = final_funds - initial_funds  # 盈亏计算
    return final_funds, profit

In [105]:
avg_profit = 0
avg_final_funds = 0
for i in range(periods):
    final_funds,profit = trade_strategy(data_list[i], initial_funds, stocks, buy_sell_points)
    avg_profit += profit
    avg_final_funds += final_funds
    
avg_profit /= periods
avg_final_funds /= periods
print("Final funds: ${}".format(avg_final_funds))
print("Profit: ${}".format(avg_profit))
# compute the profit rate
profit_rate = avg_profit / initial_funds
print("Profit rate: {:.2%}".format(profit_rate))
# get how many times we buy and sell separately
buy_sell_points_df = pd.DataFrame(buy_sell_points, columns=['time', 'action'])
print(buy_sell_points_df['action'].value_counts()/periods)

Final funds: $11512.551659573443
Profit: $1512.5516595734427
Profit rate: 15.13%
action
buy     26.6
sell    22.1
Name: count, dtype: float64


In [113]:
def trade_strategy_1020(trade_data, funds, stocks, buy_sell_points):
    stocks = 0
    for index, row in trade_data.iterrows():
        forecast = row['Forecast']
        price = row['avg_price']
        forecast20 = row['Forecast_20']
        
        if ((forecast == 0 and forecast20 == 0)) and funds >= price:
            pass
        elif (forecast == 1 and forecast20 == 1) and funds >= 5 * price:  
            stocks += 5
            funds -= 5 * price  # 更新剩余资金
            buy_sell_points.append((row['time'], 'buy'))
        elif forecast == 2 and forecast20 == 2:  
            if stocks >= 5:
                stocks -= 5
                funds += 5 * price  # 更新资金
            else:
                stocks = 0
                funds += stocks * price
            buy_sell_points.append((row['time'], 'sell'))
        elif (forecast == 0 and forecast20 == 1) and funds >= 2 * price:
            stocks += 2
            funds -= 2 * price  # 更新资金
            buy_sell_points.append((row['time'], 'buy'))
        elif (forecast == 0 and forecast20 == 2):  
            if stocks >= 2:
                stocks -= 2
                funds += 2 * price  # 更新资金
            else:
                stocks = 0
                funds += stocks * price
            buy_sell_points.append((row['time'], 'sell'))
        elif (((forecast == 1 and forecast20 == 0)
              or (forecast == 2 and forecast20 == 1)) and funds >= 1 * price):
            stocks += 1
            funds -= 1 * price  # 更新资金
            buy_sell_points.append((row['time'], 'buy'))
        elif ((forecast == 2 and forecast20 == 0) 
              or (forecast == 1 and forecast20 == 2))  and stocks >= 1:  
            stocks -= 1
            funds += 1 * price  # 更新资金
            buy_sell_points.append((row['time'], 'sell'))

    # 计算最终资金，考虑剩余的股票价值
    final_funds = funds + stocks * trade_data.iloc[-1]['avg_price']
    profit = final_funds - initial_funds  # 盈亏计算
    return final_funds, profit

In [114]:
buy_sell_points = []
avg_profit = 0
avg_final_funds = 0
for i in range(periods):
    final_funds, profit = trade_strategy_1020(data_list[i], initial_funds, stocks, buy_sell_points)
    avg_profit += profit
    avg_final_funds += final_funds
avg_profit /= periods
avg_final_funds /= periods
print("Final funds: ${}".format(avg_final_funds))
print("Profit: ${}".format(avg_profit))
# compute the profit rate
profit_rate = avg_profit / initial_funds
print("Profit rate: {:.2%}".format(profit_rate))
buy_sell_points_df = pd.DataFrame(buy_sell_points, columns=['time', 'action'])
print(buy_sell_points_df['action'].value_counts()/periods)

Final funds: $17496.243240085074
Profit: $7496.2432400850785
Profit rate: 74.96%
action
buy     37.7
sell    29.8
Name: count, dtype: float64
